# ML Model Example

This example provides a simply query tutorial for InfluxDB Cloud Dedicated. 

In [14]:
import influxdb_client_3 as InfluxDBClient3
from IPython.display import display, HTML
import pandas as pd
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

Next we provide our connection credentials to the InfluxDB 3.0 instance. 
`token` - scoped to the bucket you wish to query from.
`host` - the host of your cloud dedicated instance.
`client` - host wrapped in the flightclient class. 

In [15]:
token = ''
host = 'eu-central-1-1.aws.cloud2.influxdata.com'    
org = 'Jay-IOx'
db = 'factory'

client = InfluxDBClient3.InfluxDBClient3(token=token,
                         host=host,
                         org=org,
                         database=db) 
    


In [20]:
query = "SELECT vibration FROM machine_data WHERE \"machineID\" = 'machine1' AND time >= now() - 50m"
table = client.query(query=query, language="influxql")
print(table)

pyarrow.Table
iox::measurement: string not null
time: timestamp[ns] not null
vibration: double
----
iox::measurement: [["machine_data","machine_data","machine_data","machine_data","machine_data",...,"machine_data","machine_data","machine_data","machine_data","machine_data"]]
time: [[2023-06-20 11:45:43.107984686,2023-06-20 11:45:44.109673772,2023-06-20 11:45:45.110839057,2023-06-20 11:45:46.112391243,2023-06-20 11:45:47.114257061,...,2023-06-20 12:35:35.096919945,2023-06-20 12:35:36.098250148,2023-06-20 12:35:37.098693406,2023-06-20 12:35:38.099918373,2023-06-20 12:35:39.100949037]]
vibration: [[84,84,89,83,83,...,89,86,86,88,90]]


The `query_tools` function provides a wrapper to execution of a query agaisnt InfluxDB usinf flight. This function requires 3 paramters.
`database`- this specifies the database you wish to direct your query at.
`query` - This accepts a string literal of the query you wish to perform (SQL or InfluxQL).
`query_type` - Whether this is a SQL or InfluxQL based query.

Return all columns from the table flight for the past 3 days.

In [21]:
df = table.to_pandas()
display(df)

,iox::measurement,time,vibration
0,machine_data,2023-06-20 11:45:43.107984686,84.0
1,machine_data,2023-06-20 11:45:44.109673772,84.0
2,machine_data,2023-06-20 11:45:45.110839057,89.0
3,machine_data,2023-06-20 11:45:46.112391243,83.0
4,machine_data,2023-06-20 11:45:47.114257061,83.0
...,...,...,...
2989,machine_data,2023-06-20 12:35:35.096919945,89.0
2990,machine_data,2023-06-20 12:35:36.098250148,86.0
2991,machine_data,2023-06-20 12:35:37.098693406,86.0
2992,machine_data,2023-06-20 12:35:38.099918373,88.0


In [22]:
# Save the table to a Parquet file
pq.write_table(table, 'out.parquet' , version='2.6')

In [23]:
df.to_csv(index=False, 
path_or_buf='./out.csv')

In [ ]:
result = df.to_json(orient="records", path_or_buf='./out.json', date_unit ='ns')


In [ ]:
from pyarrow import orc
orc.write_table(table, 'out.orc')

In [ ]:
from pyarrow import feather
feather.write_feather(table, 'out.feather')